In [ ]:
from selenium import webdriver
import undetected_chromedriver as uc
from bs4 import BeautifulSoup

import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
 
options = Options()
# options.add_argument("--headless")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option("useAutomationExtension", False)
# driver = webdriver.Chrome(options=options)
# # Set up Selenium
# options = webdriver.ChromeOptions()
options.add_extension('./CAPTCHA-Solver-auto-hCAPTCHA-reCAPTCHA-freely-Chrome-Web-Store.crx')
driver = webdriver.Chrome(options=options)
# driver = uc.Chrome(options=options)
stealth(driver,
       user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.105 Safari/537.36',
       languages=["en-US", "en"],
       vendor="Google Inc.",
       platform="Win32",
       webgl_vendor="Intel Inc.",
       renderer="Intel Iris OpenGL Engine",
       fix_hairline=True,
       )
products ={}
driver.get(f'https://www.lazada.co.id/unilever-flagship-store/?from=wangpu&langFlag=id&page=1&pageTypeId=2&q=All-Products')
time.sleep(180)
for i in range(30):
    # Navigate to the page
    driver.get(f'https://www.lazada.co.id/unilever-flagship-store/?from=wangpu&langFlag=id&page={i+1}&pageTypeId=2&q=All-Products')
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-qa-locator='product-item']"))
    )
    # Get the page source after rendering
    page_source = driver.page_source

    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    # Finding all product containers
    product_containers = soup.find_all('div', attrs={"data-qa-locator": "product-item"})

    for product in product_containers:
        # Extracting the product title
        title_element = product.find('a', href=True, title=True)
        product_title = title_element['title'] if title_element else 'Title not found'

        # Extracting the discounted price using a pattern: look for <span> with "Rp"
        discounted_price_element = product.find('span', string=lambda text: text and "Rp" in text)
        discounted_price = discounted_price_element.text if discounted_price_element else 'Discounted Price not found'

        # Extracting the price before promo: look for <del> tag
        original_price_element = product.find('del', string=lambda text: text and "Rp" in text)
        original_price = original_price_element.text if original_price_element else 'Original Price not found'

        # print(f"Product Title: {product_title}")
        # print(f"Discounted Price: {discounted_price}")
        # print(f"Original Price (Before Promo): {original_price}")
        # print('---')
        if product_title not in products and product_title != 'Title not found':
            if discounted_price == 'Discounted Price not found':
                discounted_price = None
            if original_price == 'Original Price not found':
                original_price = discounted_price
            products[product_title] = {
                'discounted_price': discounted_price,
                'original_price': original_price
            }
    for i in products:
        print(i, products[i])
    time.sleep(5)

In [ ]:
print(len(products))
print(products)

1174
{'Buavita Mini Jus Buah Asli Apel 125ml Isi 40 [Karton]': {'discounted_price': 'Rp163.600', 'original_price': 'Rp202.000'}, '[Paket isi 2] Lifebuoy Sabun Cuci Tangan Total 10 200ml': {'discounted_price': 'Rp40.200', 'original_price': 'Rp41.000'}, 'Pepsodent Toothpaste Active Remin Complex 105 G x2': {'discounted_price': 'Rp77.900', 'original_price': 'Rp108.300'}, 'Pepsodent Pasta Gigi Herbal Value Pack Untuk Nafas Segar Penghilang Bau Mulut 150g': {'discounted_price': 'Rp30.200', 'original_price': 'Rp32.100'}, 'Lifebuoy Sabun Cuci Tangan Lemon Fresh 180ml REFILL- Paket isi 2': {'discounted_price': 'Rp26.800', 'original_price': 'Rp29.300'}, 'Pepsodent Pasta Gigi Siwak & Mint 150+25 g x2': {'discounted_price': 'Rp35.200', 'original_price': 'Rp39.600'}, "Pond's Bright Miracle Ultimate Oil Control Facial Whip Foam 100g - Sabun cuci muka lembut yang dapat mengontrol minyak selama 8 jam x2": {'discounted_price': 'Rp74.400', 'original_price': 'Rp88.900'}, 'Molto Superpro Softener Pelembu

In [ ]:
import pandas as pd
promo_sku = pd.DataFrame(
    columns=["productName", "basePrice", "finalPrice"])

# Function to clean and convert price strings to integers
def clean_price(price_str):
    return int(price_str.replace("Rp", "").replace(".", "").replace(",", ""))

# Create a list to store the processed rows
data_list = []

# Process the dictionary and store the data in a list
for product_name, prices in products.items():
    base_price = clean_price(prices['original_price'])
    final_price = clean_price(prices['discounted_price'])
    
    # Append data to the list
    data_list.append({
        'productName': product_name,
        'basePrice': base_price,
        'finalPrice': final_price
    })

# Convert the list to a DataFrame and concatenate with the original DataFrame
promo_sku = pd.concat([promo_sku, pd.DataFrame(data_list)], ignore_index=True)

# Display the DataFrame
print(promo_sku)

                                            productName basePrice finalPrice
0     Buavita Mini Jus Buah Asli Apel 125ml Isi 40 [...    202000     163600
1     [Paket isi 2] Lifebuoy Sabun Cuci Tangan Total...     41000      40200
2     Pepsodent Toothpaste Active Remin Complex 105 ...    108300      77900
3     Pepsodent Pasta Gigi Herbal Value Pack Untuk N...     32100      30200
4     Lifebuoy Sabun Cuci Tangan Lemon Fresh 180ml R...     29300      26800
...                                                 ...       ...        ...
1169  Pond's Age Miracle Youthful Glow Retinol Eye C...    215000     188100
1170  Vaseline Hijab Bright Multi-Purpose Skin & Hai...    220000     107000
1171  Ponds Bright Miracle Ultimate Clarity Facial F...    903600     903600
1172  "Pond's Bright Miracle Ultimate Oil Control Fa...    919200     919200
1173  Pepsodent Pasta Gigi Pencegah Gigi Berlubang V...    598200     598200

[1174 rows x 3 columns]


In [ ]:
from datetime import datetime
promo_sku.drop_duplicates(subset=["productName"], inplace=True)
print(promo_sku)


file_name = f"../lazada/LAZADA_{datetime.now().strftime('%y%m%d')}.xlsx"
promo_sku.to_excel(file_name, index=False)


                                            productName basePrice finalPrice
0     Buavita Mini Jus Buah Asli Apel 125ml Isi 40 [...    202000     163600
1     [Paket isi 2] Lifebuoy Sabun Cuci Tangan Total...     41000      40200
2     Pepsodent Toothpaste Active Remin Complex 105 ...    108300      77900
3     Pepsodent Pasta Gigi Herbal Value Pack Untuk N...     32100      30200
4     Lifebuoy Sabun Cuci Tangan Lemon Fresh 180ml R...     29300      26800
...                                                 ...       ...        ...
1169  Pond's Age Miracle Youthful Glow Retinol Eye C...    215000     188100
1170  Vaseline Hijab Bright Multi-Purpose Skin & Hai...    220000     107000
1171  Ponds Bright Miracle Ultimate Clarity Facial F...    903600     903600
1172  "Pond's Bright Miracle Ultimate Oil Control Fa...    919200     919200
1173  Pepsodent Pasta Gigi Pencegah Gigi Berlubang V...    598200     598200

[1174 rows x 3 columns]
